<a href="https://colab.research.google.com/github/ferolaa/AI-ML-Unit-2/blob/main/Lab%20I%20-%20Data%20Anonymization/Notebook/L1_Data_Anonymization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fairness, Accountability, Transparency and Ethics Course (FATE)
## Universitat Pompeu Fabra (UPF)
### Academic Year 2025-2026
### Author: Ashwin Singh (ashwin.singh01@estudiant.upf.edu)
*** Partially based on the original exercises made by David Solans (david.solans@upf.edu) ***

Submission date: 03/02/2025 at 23:59 on Aula Global

Please work on this notebook **individually**.

# LAB 1: Data anonymization with Python

# Getting Started

## Imports

In [51]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

In [35]:
import warnings

warnings.filterwarnings(action="ignore")

## Loading Dataset
In this section we will use a synthetic dataset containing medical records of 100 users.

In [41]:
dataset_url = "https://raw.githubusercontent.com/Ashwin-19/UPF-FATE-2025/refs/heads/main/Lab%20I%20-%20Data%20Anonymization/Data/health_dataset.csv"
health_data = pd.read_csv(dataset_url)
health_data.head()

,PatientName,SSN,PatientID,Phone,Gender,BirthDate,ZipCode,Health Condition,Weight,Height
0,Pablo de Cobo,412-92-6881,1000,+34 738 644 211,Male,20011009,8003,Bird flu,80,184
1,Fabio Menendez Velasco,265-27-3243,1001,+34742032426,Male,20010110,8003,Bird flu,79,179
2,Flavia Cabañas Cabanillas,554-97-8002,1002,+34712747631,Female,20010101,8014,Bird flu,67,172
3,Eliana Vendrell-Vilalta,990-49-6929,1003,+34 712 466 137,Female,20001009,8002,Common cold and flu,93,184
4,César Mas Silva,833-83-9451,1004,+34716 22 73 14,Male,20000101,8002,Common cold and flu,66,171


## Types of Identifiers and Attributes [ 1 pts ]



**[ 0.5 pts ]** List the identifiers and quasi-identifiers in the dataset:

**[ Answer ]**
Identifiers: These are attributes that can uniquely identify an individual directly.
	•	PatientName
	•	SSN
	•	PatientID
	•	Phone

**[ 0.5 pts ]** List the confidential attributes in the dataset.

**[ Answer ]**

Quasi-Identifiers= ['Gender', 'BirthDate', ZipCode']


## Data Filtering [ 0.5 pts ]

**[ 0.5 pts ]** Remove/Suppress the columns including personally identifiable information from the dataset.

**[ Discuss ]** Why do we remove/supress them?

In [44]:
pii_columns = ["Gender",	"BirthDate",	"ZipCode",	"Health Condition",	"Weight",	"Height"]
health_data = health_data[pii_columns]
health_data.head()

,Gender,BirthDate,ZipCode,Health Condition,Weight,Height
0,Male,20011009,8003,Bird flu,80,184
1,Male,20010110,8003,Bird flu,79,179
2,Female,20010101,8014,Bird flu,67,172
3,Female,20001009,8002,Common cold and flu,93,184
4,Male,20000101,8002,Common cold and flu,66,171


# K-Anonymity

## Checking K-Anonymity [ 2 pts ]

**[ 1.5 pts ]** Create a function which checks whether the data sastifies k-anonimity given some quasi-identifiers. Also return the maximum k for which the quasi-identifiers satisfy k-anonymity.

In [48]:
def is_k_anonymous(data, quasi_identifiers):
    """
    Determines whether the data is k-anonymous quasi-identifiers
    and the maximum k for which k-anonymity is satisfied.

    Args:
        data (pandas.DataFrame): microdata
        quasi_identifiers (list): quasi-identifiers in microdata

    Returns:
        bool: indicating whether the data is k-anonymous
        int: maximum k for which k-anonymity is satisfied
    """
    # Your code here
    #return True, 1
    grouped_data = data.groupby(quasi_identifiers).size().reset_index(name='counts')
    max_k = grouped_data['counts'].max()
    k_anonymous = grouped_data['counts'].min() >= max_k

    return k_anonymous, max_k

is_k_anonymous(health_data,['Gender', 'BirthDate', 'ZipCode'])

(False, 3)

**[ 0.25 pts ]** Is the data k-anonymous for BirthDate and ZipCode? If so, what is the maximum k for which k-anonymity is satisfied?

**[ 0.25 pts ]** Is the data k-anonymous for Gender and ZipCode? If so, what is the maximum k for which k-anonymity is satisfied?

In [49]:
is_k_anonymous(health_data,['BirthDate', 'ZipCode'])

(False, 4)

## Testing K-Anonimity [ 1 pts ]

To test our function, we will use a dataset from the example given in the [K-Anonymity article](https://en.wikipedia.org/wiki/K-anonymity) on Wikipedia and satisfies 2-anonymity for any combination of the attributes ``Age``, ``Gender`` and ``State of domicile``.

In [50]:
wiki_data = pd.read_html("https://en.wikipedia.org/wiki/K-anonymity",header=0)[1]
wiki_data

,Name,Age,Gender,Height,Weight,State of domicile,Religion,Disease
0,*,20 < Age ≤ 30,Female,165 cm,72 kg,Tamil Nadu,*,Cancer
1,*,20 < Age ≤ 30,Female,162 cm,70 kg,Kerala,*,Viral infection
2,*,20 < Age ≤ 30,Female,170 cm,68 kg,Tamil Nadu,*,Tuberculosis
3,*,20 < Age ≤ 30,Male,170 cm,75 kg,Karnataka,*,No illness
4,*,20 < Age ≤ 30,Female,165 cm,71 kg,Kerala,*,Heart-related
5,*,20 < Age ≤ 30,Male,160 cm,69 kg,Karnataka,*,Tuberculosis
6,*,Age ≤ 20,Male,167 cm,85 kg,Kerala,*,Cancer
7,*,20 < Age ≤ 30,Male,180 cm,81 kg,Karnataka,*,Heart-related
8,*,Age ≤ 20,Male,175 cm,79 kg,Kerala,*,Heart-related
9,*,Age ≤ 20,Male,169 cm,82 kg,Kerala,*,Viral infection


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Age').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Gender').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Height').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Disease'].value_counts()
    for x_label, grp in _df_13.groupby('State of domicile')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('State of domicile')
_ = plt.ylabel('Disease')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Age'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='index', y='Age', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Gender'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='index', y='Gender', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['Height'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='index', y='Height', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['State of domicile'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='index', y='State of domicile', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

**[ 1 pts ]** Test k-anonimity for the three attributes and their different combinations picking two attributes at a time. For each combination, find the maximum k satisfying k-anonymity.

In [66]:
attributes = ['Age', 'Gender', 'State of domicile']

# Testing different combinations
results = {}
for combination in combinations(attributes, 2):
    k_anonymous, max_k = is_k_anonymous(wiki_data, list(combination))
    results[combination] = (k_anonymous, max_k)

results


{('Age', 'Gender'): (False, 4),
 ('Age', 'State of domicile'): (False, 3),
 ('Gender', 'State of domicile'): (False, 3)}

## Properties of K-Anonymity [ 1 pts ]
> You may use examples from the previous exercises.

**[ 0.5 pts ]** If a dataset is $k_1$-anonymous for a set of attributes, and $k_2$-anonymous for any subset of those attributes, what is the relationship between $k_1$ and $k_2$? Can you show an example?

**[ Answer ]**
	•	k1 is the level of anonymity when considering multiple attributes together.
	•	k2 is the level of anonymity for any smaller group of those attributes.

So, k2 can never be higher than k1 because subsets have fewer attributes to group by.

Example:
	•	If a dataset is k1-anonymous for Age, Gender, and State of domicile (meaning every group has at least 3 individuals),
	•	Then for smaller combinations like Age and Gender, the dataset might be k2-anonymous, but k2 can’t be higher than 3 (in this case).

In our example:
	•	k1 = 3 (for Age, Gender, and State of domicile),
	•	k2 = 4 (for Age and Gender),
	•	So, k2 ≤ k1.

**[ 0.5 pts ]** If a dataset is $k_1$-anonymous given an attribute $a_1$, and $k_2$-anonymous given an attribute $a_2$, is the dataset always $k$-anonymous given attributes $\{a_1,a_2\}$ ? If not, show an example, and find the maximum $k$ for which the dataset can be $k$-anonymous.

**[ Answer ]**

## Methods for K-Anonymity [ 5 pts ]

### GLOBAL Recoding (Generalization)

**[ 0.25 pts ]** Create a function which applies **global recoding** to an attribute in the data upto $n$ digits.

In [70]:
def global_recoding(data, attribute, n):
    """
    applies global recoding to an attribute in the data for n digits

    Args:
        data (pandas.DataFrame): data for anonymization
        attribute (str): attribute to be recoded
        n (int): number of digits to be recoded

    Returns:
        pandas.DataFrame: globally recoded data
    """
    if attribute in data.columns:
        data[attribute] = data[attribute].astype(str).str[:n]
    else:
        raise ValueError(f"Attribute '{attribute}' not found in the dataset.")
    return data
global_recoding(wiki_data, "Age", 1)

,Name,Age,Gender,Height,Weight,State of domicile,Religion,Disease
0,*,2,Female,165 cm,72 kg,Tamil Nadu,*,Cancer
1,*,2,Female,162 cm,70 kg,Kerala,*,Viral infection
2,*,2,Female,170 cm,68 kg,Tamil Nadu,*,Tuberculosis
3,*,2,Male,170 cm,75 kg,Karnataka,*,No illness
4,*,2,Female,165 cm,71 kg,Kerala,*,Heart-related
5,*,2,Male,160 cm,69 kg,Karnataka,*,Tuberculosis
6,*,A,Male,167 cm,85 kg,Kerala,*,Cancer
7,*,2,Male,180 cm,81 kg,Karnataka,*,Heart-related
8,*,A,Male,175 cm,79 kg,Kerala,*,Heart-related
9,*,A,Male,169 cm,82 kg,Kerala,*,Viral infection


**[ 0.75 pts ]** Use the function to list all possible ways of 3-anonymizing the dataset given attributes ``BirthDate`` and ``ZipCode``. What is the best way of 3-anonymizing the dataset if we want to preserve maximum information about the health conditions across different zip codes?

**[ Answer ]**

In [ ]:
# your code here

**[ 1 pts ]** For the best way of $3$-anonymizing the dataset from last question, comment on the sufficiency of $k$-anonymity against **homogeneity** and **internal** attacks aimed at inferring the health condition of patients. If you find $k$-anonymity to be insufficient, you must show quasi-identifier equivalence classes (groups) susceptible to the attacks.

In [ ]:
# your code here

In [ ]:
# your code here

### Microaggregation

**[ 2 pts ]** Create a function which microaggregates continuous attributes (``height, weight``) to achieve $k$-anonymity, while minimizing the loss in information across both attributes. Return the best-possible $k$-anonymity and corresponding loss.

**Recommended Approach**: Experiment with a clustering algorithm for $n \leq \frac{|Data|}{k}$ clusters to achieve $k$-anonymity, and use sum of squared errors between original and anonymized data for loss of information.

In [ ]:
def microaggregation(data,attributes,k):
    """
    applies microaggregation to attributes in the data
    to achieve k'-anonymity, k' >= k

    Args:
        data (pandas.DataFrame): data for anonymization
        attributes (list): attributes to be microaggregated
        k (int): lower bound of k-anonymity to achieve

    Returns:
        tuple:
            float: loss of information to achieve k' anonymity
            int: k' corresponding to k'-anonymity achieved
    """
    return loss, observed_k

**[ 1 pts ]** Plot the relationship between $k$-anonymity achieved v/s loss of information for $k \in \{2,3,4,...10\}$. What do you observe?

In [ ]:
# your code here

# L-Diversity

## Checking L-Diversity [ 1.5 pts ]

**[ 0.5 pts ]** Create a function which checks whether a dataset is l-diverse given a sensitive attribute and quasi-identifiers. If so, return the maximum-l for which the dataset satisfies l-diversity.

In [ ]:
def is_l_diverse(data, sensitive, quasi_identifiers):
    """
    checks whether a data is l-diverse for a sensitive
    attribute given quasi-identifiers, and returns the
    maximum l for which the data satisfies l-diversity

    Args:
        data (pandas.DataFrame): data to check l-diversity for
        sensitive (str): sensitive attribute for l-diversity
        quasi_identifiers (list): list of quasi-identifiers

    Returns:
        tuple:
            bool: indicating whether the data is l-diverse
            int: maximum l for which the data satisfies l-diversity
    """
    return True, l

> Answer the following questions for the original dataset, globally recoded dataset.

**[ 0.5 pts ]** Is the dataset l-diverse given ``Health Condition`` as sensitive attribute and ``ZipCode, BirthDate`` as quasi-identifiers?

In [ ]:
# your code here

In [ ]:
# your code here

**[ 0.5 pts ]** Is the dataset l-diverse given ``Gender`` as sensitive attribute and ``ZipCode, BirthDate`` as quasi-identifiers?

In [ ]:
# your code here

In [ ]:
# your code here

## L-Diversity via Local Suppression [ 2 pts ]

**[ 1 pts ]** Create a function to apply local suppression to groups within a $k$-anonymized dataset which violate $l$-diversity given a sensitive attribute, two quasi-identifiers, and $l$. If the same is not feasible, raise an error.


In [ ]:
def local_suppression(data,sensitive,quasi_identifiers,l):
    """
    locally suppresses quasi-identifier equivalence classes
    in anonymized data which violate l-diversity so that the
    overall data satisfies l-diversity

    Args:
        data (pandas.DataFrame): data for local suppression
        sensitive (str): sensitive attribute for l-diversity
        quasi_identifiers (list): quasi-identifiers to suppress
        l (int): l-diversity constraint to satisfy

    Returns:
        pandas.DataFrame: locally suppressed data satisfying l-diversity
    """
    # your code here
    return data

**[ 0.5 pts ]** Using the function, make the globally recoded dataset $2$-diverse for sensitive attribute ``Health Condition`` given quasi_identifiers ``ZipCode``,``BirthDate``, and show the suppressed groups.

In [ ]:
# your code here

In [ ]:
# your code here

**[ 0.5 pts ]** Using the function, make the globally recoded dataset $3$-diverse for sensitive attribute ``Gender`` given quasi_identifiers ``ZipCode``,``BirthDate``, and show the suppressed groups.

In [ ]:
# your code here

In [ ]:
# your code here

## Sufficiency of $k$-anonymity and $l$-diversity [ 1 pts ]

**[ 1 pts ]** Describe one scenario where $k$-anonymity and $l$-diversity would be sufficient for guaranteeing the anonymity of a dataset, and one scenario where they will be insufficient. You may use tables to illustrate your answer.

**[ Answer ]**